In [1]:
!pip install -U pyautogen openai dataset -qq

In [45]:
import autogen
from autogen import ConversableAgent


llm_config={
    "model": "gpt-35-turbo",
    "api_type": "azure",
    "api_key": "13d8d0888405404b9c1ee4407ad19226",
    "base_url": "https://openai-resource-for-multilingual.openai.azure.com/",
    "api_version": "2023-07-01-preview",
  }




In [46]:
machine_translation_agent = ConversableAgent(
    name="translation agent",
    system_message="""You are a helpful agent which is capable of doing machine translation 
    from any language to english,
    """,
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [47]:
sentiment_analysis_agent = ConversableAgent(
    name="Sentiment analysis agent",
    system_message="""You are a helpful agent given english text you can classify 
    into one of the sentiment classes. the options are 'positive' 'negative' or 'neutral'
    """,
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [48]:
output_agent = ConversableAgent(
    name="Output Agent",
    system_message="""You are a helpful sentiment agent that tells the 
            the customer the setiment of their text and clear explanation why
            """,
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [49]:
input_agent = ConversableAgent(
    name="input_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [53]:
chats = [
    {
        "sender": machine_translation_agent,
        "recipient": input_agent,
        "message": 
            "Hello, I'm here to help you identify sentiment of a text from any language",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the translaton"
                             "into as JSON object only: "
                             "{'text': '', 'translation': ''}",
        },
        "max_turns": 1,
        "clear_history" : True
    },
    {
        "sender": machine_translation_agent, 
        "recipient": sentiment_analysis_agent,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the translaton"
                             "into as JSON object only: "
                             "{'text': '', 'sentiment': ''}",
        },
        "max_turns": 1,
        "clear_history" : True
    },
    {
        "sender": sentiment_analysis_agent,
        "recipient": output_agent,
        "message": "here is what we know about your text.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

In [56]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)



********************************************************************************
Starting a new chat....

********************************************************************************
translation agent (to input_agent):

Hello, I'm here to help you identify sentiment of a text from any language

--------------------------------------------------------------------------------
input_agent (to translation agent):

ሰላም ለሰው

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
translation agent (to Sentiment analysis agent):

ሰላም ለሰው
Context: 
Sure, I can help with that. Can you please provide the text you want to be translated?
Sure! Here is the translation of "ሰላም ለሰው" to English: 

{'text': 'ሰላም ለሰው', 'translation': 'Hello to the person'}

--------------------------------

In [57]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")
    

Sure! Here is the translation of "ሰላም ለሰው" to English: 

{'text': 'ሰላም ለሰው', 'translation': 'Hello to the person'}


Sure! Here is the JSON object with the translation and sentiment:

{'text': 'ሰላም ለሰው', 'sentiment': 'Neutral'}


The given text "ሰላም ለሰው" translates to "Hello to the person" in English and the sentiment analysis output of the text is "Neutral".


